In [9]:
import os
import pandas as pd
from pathlib import Path
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'


In [10]:
# acquire system path of the current file, join it with the related path of the csv file path.
increase_path = "../Data/Increase/etfs-increase-change-in-open-interest-11-13-2023.csv"
decrease_path = "../Data/Decrease/etfs-decrease-change-in-open-interest-11-13-2023.csv"

#read csv files
increase = pd.read_csv(Path(increase_path))
decrease = pd.read_csv(Path(decrease_path))

print("Increase DataFrame column types:")
print(increase.dtypes)

print("\nDecrease DataFrame column types:")
print(decrease.dtypes)



Increase DataFrame column types:
Symbol       object
Price~      float64
Type         object
Strike      float64
Exp Date     object
DTE         float64
Bid         float64
Ask         float64
Last        float64
Volume      float64
Open Int    float64
OI Chg       object
Delta       float64
IV           object
Time         object
dtype: object

Decrease DataFrame column types:
Symbol       object
Price~      float64
Type         object
Strike      float64
Exp Date     object
DTE         float64
Bid         float64
Ask         float64
Last        float64
Volume      float64
Open Int    float64
OI Chg       object
Delta       float64
IV           object
Time         object
dtype: object


In [11]:
# Convert 'Strike' column to float64 in the Decrease DataFrame
increase[['OI Chg', 'Strike']] = increase[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)
decrease[['OI Chg', 'Strike']] = decrease[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)

#combine increase and decrease data
combine_df = increase.merge(decrease, how= 'outer')

DTE = "max"
#Select the DTE
if DTE == 'min':
    combine_min_DTE = combine_df[combine_df['DTE'] == combine_df['DTE'].min()]
elif DTE == 'max':
    combine_min_DTE = combine_df[combine_df['DTE'] <= combine_df['DTE'].max()]
else:
    combine_min_DTE = combine_df[combine_df['DTE'] <= DTE]

#Clean the NA data
combine_min_DTE = combine_min_DTE.dropna()

# Transfer columns 'OI Chg' and 'Strike' datatype from str to float.
combine_min_DTE[['OI Chg', 'Strike']] = combine_min_DTE[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)

#Transfer column 'IV'datatype from str to float.
combine_min_DTE['IV'] = combine_min_DTE['IV'].str.replace('%', '').str.replace(',', '').astype(float)/100

#Sorted by orders 'Symbol','Type','Strike','Open Int','Volume','OI Chg','IV'.
combine_sort_df = combine_min_DTE.sort_values(['Symbol','Type','Strike','Open Int','Volume','OI Chg','IV'])
combine_sort_df.columns = ['Symbol','Price','Type','Strike','Exp Date','DTE','Bid','Midpoint','Ask','Last','Volume','Open Int','OI Chg','IV','Time']

# return data
combine_sort_df

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,OI Chg,IV,Time
1321,ARKG,23.740,Call,28.0,2023-12-15,32.0,0.10,0.20,0.15,500.0,4083.0,-4.0,0.111012,0.4173,10:38 ET
316,ARKG,23.740,Put,23.5,2023-12-01,18.0,0.70,0.80,0.95,1004.0,6049.0,1045.0,-0.423661,0.4339,10:54 ET
1084,ARKG,23.740,Put,24.0,2023-12-15,32.0,1.25,1.35,1.55,1000.0,3623.0,-432.0,-0.491695,0.4401,10:41 ET
1750,ARKK,39.800,Call,39.5,2023-11-24,11.0,1.14,1.18,1.13,569.0,564.0,59.0,0.575453,0.3484,15:09 ET
536,ARKK,39.800,Call,40.0,2023-11-17,4.0,0.58,0.61,0.59,4069.0,10191.0,553.0,0.470905,0.3994,15:12 ET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,YINN,26.520,Call,28.5,2023-11-17,4.0,0.35,0.45,0.39,1096.0,550.0,197.0,0.255217,0.9632,15:07 ET
1642,YINN,26.520,Call,29.0,2023-11-17,4.0,0.30,0.35,0.35,1305.0,370.0,19.0,0.222417,1.0370,13:31 ET
683,YINN,26.520,Put,25.0,2023-11-17,4.0,0.40,0.50,0.45,502.0,1846.0,372.0,-0.261959,0.9695,13:39 ET
1472,ZEB.TO,31.125,Call,34.0,2024-03-15,123.0,0.07,0.17,0.15,4000.0,35218.0,0.0,0.129380,0.1321,13:29 ET


In [12]:
import sys
import os
import psycopg2
import psycopg2.extras
import pandas as pd
import pytz
from datetime import datetime
import traceback

# Load and assign variables.

SQL_DB = ''
SQL_USER = ''
SQL_PASSWORD = ''
SQL_HOST = ''

def connect_data_base(database=SQL_DB, user=SQL_USER, password=SQL_PASSWORD, host=SQL_HOST):
    con = psycopg2.connect(database=SQL_DB, user=SQL_USER, password=SQL_PASSWORD, host=SQL_HOST)
    cur = con.cursor()
    cur.execute('SELECT version()')
    version = cur.fetchone()[0]
    print(f"DataBaseFlow Connected!\nVersion: {version}")
    return con, cur

date = "11-13-2023"

now_timestamp = int(datetime.now().timestamp()) #时间戳只保留秒（去掉更低的精度，保留整数部分）
date_parts = date.split('-')  #分割日期字符串
con, cur = connect_data_base() #连接数据库



DataBaseFlow Connected!
Version: PostgreSQL 14.8 on aarch64-unknown-linux-musl, compiled by gcc (Alpine 12.2.1_git20220924-r10) 12.2.1 20220924, 64-bit


In [13]:
combine_data = combine_sort_df

# 自定义函数，根据"Last"和"Midpoint"的关系判断"sentiment"
def get_initiator(row):
    if row['Last'] < row['Bid']:
        return 'Aggressive Seller'
    elif row['Last'] > row['Ask']:
        return 'Aggressive Buyer'
    elif row['Last'] < row['Midpoint']:
        return 'Seller'
    elif row['Last'] > row['Midpoint']:
        return 'Buyer'
    else:
        return 'Neutral'
# Add "Initiator" column
combine_data['Initiator'] = combine_data.apply(get_initiator, axis=1)

#重新排列，把'Initiator'放在'Last'后：
new_order = ['Symbol', 'Price', 'Type', 'Strike', 'Exp Date', 'DTE', 'Bid', 'Midpoint', 'Ask', 'Last', 'Initiator', 'Volume', 'Open Int', 'OI Chg', 'IV', 'Time']

# 使用reindex()方法重新排列列的顺序
combine_data = combine_data.reindex(columns=new_order)

#构建写操作的表名
write_table_name = f"_{date_parts[2]}_{date_parts[0]}_{date_parts[1]}_etfs_{now_timestamp}"  # 按照'YYYY_MM_DD'格式重新排列日期部分

# Delete old data from the table if it exists
delete_query = f"DROP TABLE IF EXISTS {write_table_name}"
cur.execute(delete_query)
con.commit()

# Create table if not exists
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {write_table_name} (
    "Symbol" TEXT,
    "Price" NUMERIC,
    "Type" TEXT,
    "Strike" NUMERIC,
    "Exp Date" TEXT,
    "DTE" NUMERIC,
    "Bid" NUMERIC,
    "Midpoint" NUMERIC,
    "Ask" NUMERIC,
    "Last" NUMERIC,
    "Initiator" TEXT,
    "Volume" NUMERIC,
    "Open Int" NUMERIC,
    "OI Chg" NUMERIC,
    "IV" NUMERIC,
    "Time" TEXT
)
"""
cur.execute(create_table_query)
con.commit()

# 批量插入数据
insert_query = f"INSERT INTO {write_table_name} VALUES %s"
rows = [tuple(row) for _, row in combine_data.iterrows()]
psycopg2.extras.execute_values(cur, insert_query, rows)
con.commit()
print("Data inserted successfully!")
print(f"{write_table_name}")

Data inserted successfully!
_2023_11_13_etfs_1700031712
